# 在Kaggle上运行LLMs训练项目的详细步骤
本notebook提供了在Kaggle环境中运行LLMs训练项目的完整、详细流程。每个步骤都包含具体的代码实现和必要的说明。
## 1. 环境准备
首先，我们需要克隆项目代码并安装所需的依赖项。所有依赖项都在`requirements.txt`中定义。

In [ ]:
# 克隆项目代码
!git clone https://github.com/your-username/llmscopy.git
!cd llmscopy

# 安装依赖项
!pip install torch>=1.10.0 torchvision>=0.11.0 torchaudio>=0.10.0
!pip install transformers>=4.15.0 datasets>=1.18.0 tokenizers>=0.10.3
!pip install tensorboard>=2.7.0 wandb>=0.12.0 tqdm>=4.62.0
!pip install scikit-learn>=1.0.0 numpy>=1.20.0 pandas>=1.3.0

import os
os.chdir('llmscopy')

## 2. 数据准备
### 2.1 数据格式说明
训练数据应位于`data/processed/train.txt`，验证数据位于`data/processed/val.txt`。
数据文件的格式要求：
- 每行一个训练样本
- 文本应该是预处理过的（清理过的）
- 编码格式为UTF-8

### 2.2 设置数据路径

In [ ]:
# 设置数据路径
TRAIN_DATA_PATH = '/kaggle/input/your-dataset/train.txt'
VAL_DATA_PATH = '/kaggle/input/your-dataset/val.txt'

# 检查数据文件
!head -n 5 $TRAIN_DATA_PATH
print(f'训练数据行数: {!wc -l $TRAIN_DATA_PATH}')

## 3. 配置模型和训练参数
### 3.1 加载和修改配置
我们使用`configs/model_config.py`中定义的配置，并根据Kaggle环境进行必要的调整。

In [ ]:
from configs.model_config import get_small_config

config = get_small_config()

# 1. 路径配置
config.train_file = TRAIN_DATA_PATH
config.val_file = VAL_DATA_PATH
config.output_dir = '/kaggle/working/checkpoints'

# 2. 训练参数配置
config.batch_size = 32  # 根据GPU显存调整
config.gradient_accumulation_steps = 4  # 梯度累积步数
config.max_steps = 50000  # 总训练步数
config.warmup_steps = 2000  # 预热步数

# 3. 优化器配置
config.learning_rate = 5e-5
config.weight_decay = 0.01
config.adam_beta1 = 0.9
config.adam_beta2 = 0.999
config.adam_epsilon = 1e-8

# 4. 混合精度训练配置
config.fp16 = True  # 启用混合精度训练

print('配置信息:
', config)

## 4. 训练流程
### 4.1 数据加载和预处理
使用`src/data/preprocessing.py`中的`TextDataset`类加载和处理数据。

In [ ]:
from src.data.preprocessing import TextDataset
from src.data.tokenizer import get_tokenizer

# 1. 获取分词器
tokenizer = get_tokenizer()

# 2. 创建训练集
train_dataset = TextDataset(
    file_path=config.train_file,
    tokenizer=tokenizer,
    block_size=config.n_positions,
    shuffle=True
)

# 3. 创建验证集
val_dataset = TextDataset(
    file_path=config.val_file,
    tokenizer=tokenizer,
    block_size=config.n_positions,
    shuffle=False
)

print(f'训练集大小: {len(train_dataset)}')
print(f'验证集大小: {len(val_dataset)}')

### 4.2 初始化模型和训练器
使用`src/models/transformer.py`中的`GPT`模型和`src/training/trainer.py`中的`Trainer`类。

In [ ]:
from src.models.transformer import GPT
from src.training.trainer import Trainer

# 1. 初始化模型
model = GPT(config)

# 2. 打印模型信息
total_params = sum(p.numel() for p in model.parameters())
print(f'模型总参数量: {total_params:,}')

# 3. 初始化训练器
trainer = Trainer(
    model=model,
    config=config,
    train_dataset=train_dataset,
    val_dataset=val_dataset
)

### 4.3 开始训练
训练过程将自动执行以下操作：
1. 每10步记录训练损失
2. 每1000步进行验证
3. 每5000步保存检查点
4. 保存验证损失最低的模型

In [ ]:
# 开始训练
global_step, best_val_loss = trainer.train()

print(f'训练完成！')
print(f'总训练步数: {global_step}')
print(f'最佳验证损失: {best_val_loss:.4f}')

## 5. 保存和导出结果
训练结果将保存在以下位置：
- 模型检查点：`/kaggle/working/checkpoints/checkpoint-{step}/`
- 最佳模型：`/kaggle/working/checkpoints/best_model/`
- 最终模型：`/kaggle/working/checkpoints/final_model/`
- TensorBoard日志：`/kaggle/working/runs/`

### 重要提示
1. 确保在Kaggle notebook设置中启用GPU
2. 定期下载重要的checkpoint，因为Kaggle notebook重启后数据会丢失
3. 可以使用TensorBoard查看训练过程中的损失曲线和学习率变化